## CSV Data to MySQL for use in VISDOM

This notebook can be used to construct an 'account' table and a 'meter_data' table in mysql, based on the csv files with data extracted from the prop39schools xml files, and an 'intervention' table based on the PEPS_Data.xlsx file available on the [prop39 data site](http://www.energy.ca.gov/efficiency/proposition39/data/).

The processed csv files can be downloaded as a zip file from [here](https://s3.amazonaws.com/BrianCoffey/PGE_csv_2.zip). They should be unzipped locally before running this script. Note that you may also need to `pip install xlrd` to be able to run pandas.read_excel for the intervention table data.

You must first create a database in mysql with your desired name (e.g. visdom_data_PGE), and create a data_db.cfg file to point to it, as described in the next section below.

The script will likely take about 40 minutes to complete on most modern laptops.

For your database to be ready for use in VISDOM, you will also need to load a local_weather table into your database. To do so you can follow the instructions in the [local-weather repo](https://github.com/sborgeson/local-weather) and use the prop39_config.csv file found [here](https://s3.amazonaws.com/BrianCoffey/prop39_weather_config.csv), which was prepared using the accompanying notebook. Or, if you want to get to the end point faster, you can download [this csv file](https://s3.amazonaws.com/BrianCoffey/prop39_weather_dump.csv), which was constructed using that repo, and then simply modify [this sql query](https://github.com/sborgeson/local-weather/blob/master/import_weather_data_mysql.sql) to point to that csv file and run it. Running the sql script will take about 20 minutes.

Once the database is set up, you can set it up as a DATA_SOURCE in VISDOM with the accompanying prop39_visdom_data_source.R file and test it via the sanitycheck function as follows:

```
source("prop39_visdom_data_source.R")
DATA_SOURCE = MyDataSource()
sanityCheckDataSource(DATA_SOURCE)
```

As described in more detail in the ID_mapping notebook in the same folder as this notebook, an account_uuid generally corresponds with an individual school and may have one or multiple meters associated with it, and may also have multiple interventions (or no interventions) associated with it.

### File locations, database config

Point this to the directory with the meter data csv files:

In [1]:
csv_dir = "PGE_csv"

Read your database connection details from a data_db.cfg file with the following format:

```
dbType=MySQL
user=[database user]
pass=[password (if applicable)]
db=[name of database]
```

In [2]:
db_pass = ""
with open('data_db.cfg','r') as f:
    for line in f:
        s = line.split("=")
        if s[0].strip() == "user":
            db_user = s[1].strip()
        if s[0].strip() == "pass":
            db_pass = ":" + s[1].strip()
        if s[0].strip() == "db":
            db_db = s[1].strip()

### Notebook config

In [3]:
import pandas as pd
import numpy as np
import mysql.connector, os, datetime
from sqlalchemy import create_engine

In [4]:
engine = create_engine('mysql+mysqlconnector://' + db_user + db_pass + '@localhost/' + db_db, echo=False)
conn = engine.connect()

### Reading the account table data from the _BILL.csv files

In [5]:
usecols = [
    'utility',
    'customer_name',
    'customer_city',
    'customer_zip',
    'customer_account',
    'lea_customer',
    'cds_code',
    'school_site_name',
    'school_city',
    'school_site_zip',
    'agreement',
    'rate_schedule_id'    
]

In [6]:
accounts_list = []
for root, dirs, files in os.walk(csv_dir):
    for f in files:
        if f.endswith('_BILL.csv'):
            df = pd.read_csv(os.path.join(root,f), usecols=usecols)
            df = df.drop_duplicates()
            accounts_list.extend(df.to_dict(orient='records'))

accounts_df = df.from_records(accounts_list)
accounts_df = accounts_df.drop_duplicates()
print len(accounts_df)
accounts_df.head(3)

4872


,agreement,cds_code,customer_account,customer_city,customer_name,customer_zip,lea_customer,rate_schedule_id,school_city,school_site_name,school_site_zip,utility
0,8496493494,1100170130419,NaN,HAYWARD,ALAMEDA COUNTY OFFICE OF EDUCATION,94544-1136,NaN,E19S,Hayward,Alameda County Community,94544-1136,PacificGasElectric
1,8496493494,1100176106751,NaN,HAYWARD,ALAMEDA COUNTY OFFICE OF EDUCATION,94544-1136,NaN,E19S,Hayward,Alameda County Special Education,94544-1136,PacificGasElectric
2,5637199749,1612340000000,NaN,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,NaN,HA10SX,Newark,LEA,94560-0385,PacificGasElectric


In [7]:
accounts_df.columns.tolist()

['agreement',
 'cds_code',
 'customer_account',
 'customer_city',
 'customer_name',
 'customer_zip',
 'lea_customer',
 'rate_schedule_id',
 'school_city',
 'school_site_name',
 'school_site_zip',
 'utility']

In [8]:
accounts_df.columns = [
 'meter_uuid',
 'account_uuid',
 'customer_account',
 'customer_city',
 'customer_name',
 'customer_zip',
 'lea_customer',
 'rate_schedule_id',
 'school_city',
 'school_site_name',
 'school_site_zip',
 'utility_name'
]

In [9]:
accounts_df['zip5'] = accounts_df['school_site_zip'].str[:5]
accounts_df.head(3)

,meter_uuid,account_uuid,customer_account,customer_city,customer_name,customer_zip,lea_customer,rate_schedule_id,school_city,school_site_name,school_site_zip,utility_name,zip5
0,8496493494,1100170130419,NaN,HAYWARD,ALAMEDA COUNTY OFFICE OF EDUCATION,94544-1136,NaN,E19S,Hayward,Alameda County Community,94544-1136,PacificGasElectric,94544
1,8496493494,1100176106751,NaN,HAYWARD,ALAMEDA COUNTY OFFICE OF EDUCATION,94544-1136,NaN,E19S,Hayward,Alameda County Special Education,94544-1136,PacificGasElectric,94544
2,5637199749,1612340000000,NaN,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,NaN,HA10SX,Newark,LEA,94560-0385,PacificGasElectric,94560


In [10]:
reals = accounts_df[['account_uuid']].applymap(np.isreal)
accounts_df = accounts_df[reals['account_uuid']]
len(accounts_df)

4864

In [11]:
accounts_df = accounts_df.drop_duplicates(subset=['meter_uuid'], keep='last')
len(accounts_df)

3177

In [12]:
accounts_df = accounts_df.dropna(subset=['zip5'])
len(accounts_df)

2575

### Creating the account table in the desired format and writing to it

In [13]:
create_table_sql = '''
CREATE TABLE `account` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `account_uuid` varchar(20) DEFAULT NULL,
  `meter_uuid` varchar(20) DEFAULT NULL,
  `zip5` varchar(5) DEFAULT NULL,
  `customer_account` varchar(50) DEFAULT NULL,
  `customer_city` varchar(50) DEFAULT NULL,
  `customer_name` varchar(50) DEFAULT NULL,
  `customer_zip` varchar(10) DEFAULT NULL,
  `lea_customer` varchar(50) DEFAULT NULL,
  `rate_schedule_id` varchar(50) DEFAULT NULL,
  `school_city` varchar(50) DEFAULT NULL,
  `school_site_name` varchar(100) DEFAULT NULL,
  `school_site_zip` varchar(10) DEFAULT NULL,
  `utility_name` varchar(50) DEFAULT NULL,
  PRIMARY KEY (`id`),
  KEY `zip5_meter_uuid_idx` (`METER_UUID`,`zip5`),
  KEY `account_uuid_idx` (`ACCOUNT_UUID`),
  KEY `meter_uuid_idx` (`METER_UUID`)
)
'''

conn.execute('DROP TABLE IF EXISTS `account`;')
conn.execute(create_table_sql)

In [14]:
accounts_df.to_sql(name='account', con=engine, if_exists='append', index=False)

Quick test to make sure it's working:

In [15]:
pd.read_sql('SELECT * FROM account LIMIT 3;', con=engine)

,id,account_uuid,meter_uuid,zip5,customer_account,customer_city,customer_name,customer_zip,lea_customer,rate_schedule_id,school_city,school_site_name,school_site_zip,utility_name
0,1,1100176106751,8496493494,94544,None,HAYWARD,ALAMEDA COUNTY OFFICE OF EDUCATION,94544-1136,None,E19S,Hayward,Alameda County Special Education,94544-1136,PacificGasElectric
1,2,1612340000000,5637199749,94560,None,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,None,HA10S,Newark,LEA,94560-0385,PacificGasElectric
2,3,1612340130054,5637199200,94560,None,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,None,A10S,Newark,Newark Memorial High,94560-5007,PacificGasElectric


In [16]:
pd.read_sql('SELECT COUNT(*) FROM account;', con=engine)

,COUNT(*)
0,2575


In [17]:
pd.read_sql('SELECT COUNT(DISTINCT(meter_uuid)) FROM account;', con=engine)

,COUNT(DISTINCT(meter_uuid))
0,2575


### Creating the intervention table

The following will read the interventions data from the PEPS_data.xlsx file, add the appropriate account_uuid to each entry (if applicable, Null otherwise) by left-merging with the accounts_df table, then edits column names for a few columns to keep them under the requisite 64 characters, and then writes it to a mysql table. 

In [ ]:
accounts_df['site_pair'] = accounts_df['school_city'] + "_" + accounts_df['school_site_name']

interventions_df = pd.read_excel('PEPS_Data.xlsx', sheetname='Data- Approved EEPs')

interventions_df['site_pair'] = interventions_df['Site City'] + "_" + interventions_df['Site Name']
interventions_df = pd.merge(interventions_df,accounts_df[['site_pair','account_uuid']],how='left',on='site_pair')
del interventions_df['site_pair']

replacement_column_names = [
    'Grant Amount Req Based on Single or Multiple Years Allocation',
    'Grant Amount Req',
    'Were Planning Funds Requested from CA Department of Education',
    'Budget for Screening and Energy Audits Over Program Life',
    'Budget for Prop 39 Program Assistance Over Program Life',
    'Est First Yr Annual Electricity Production of PV Measure',
    'Est Total Rebates Plus Oth Non-Repayable Funds for PV Measure',
    'Est First Year Elec Prod of PPA Measure Generation System',
    'Est PPA Measure Elec Gen as Percent of Baseline Elec Usage'
]

k = 0
for j,i in enumerate(interventions_df.columns):
    if len(i) > 64:
        interventions_df.columns.values[j] = replacement_column_names[k]
        k += 1

conn.execute('DROP TABLE IF EXISTS `intervention`;')
interventions_df.to_sql(name='intervention', con=engine, chunksize=100)
conn.execute('ALTER TABLE intervention MODIFY account_uuid VARCHAR(20);')

### Creating the meter_data table in the desired format

Note: this is currently treating the meter_uuid, account_uuid, date and zip5 as integers, but they should more likely be treated as varchar, varchar, datetime and varchar, respectively.

In [18]:
create_table_sql = '''
CREATE TABLE `meter_data` (
  `meter_uuid` varchar(20) NOT NULL,
  `account_uuid` varchar(20) NOT NULL,
  `date` DATE NOT NULL,
  `zip5` varchar(5) DEFAULT NULL,
'''

for i in range(1,97):
    create_table_sql += "`h" + str(i) + "` int(11) DEFAULT NULL,\n"

create_table_sql += '''    
  PRIMARY KEY (`meter_uuid`,`date`),
  KEY `meter_uuid_idx` (`meter_uuid`),
  KEY `account_uuid_idx` (`account_uuid`),
  KEY `zip_Date_idx` (`date`,`zip5`),
  KEY `zip_idx` (`zip5`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1;
'''

conn.execute('DROP TABLE IF EXISTS `meter_data`;')
conn.execute(create_table_sql)

### Fill the meter table with data from the csv files

In [19]:
usecols = ['agreement', 'start']
for i in range(1,97):
    usecols.append('d' + str(i))

In [20]:
colnames = ['meter_uuid', 'date']
for i in range(1,97):
    colnames.append('h' + str(i))

In [21]:
for root, dirs, files in os.walk(csv_dir):
    for f in files:
        if f.endswith('_INTERVAL.csv'):
            df = pd.read_csv(os.path.join(root,f), usecols=usecols)
            if len(df) > 0:
                df.columns = colnames
                df = df.drop_duplicates()
                for i in range(1,97):
                    df['h' + str(i)] = df['h' + str(i)] * 1000
                df = pd.merge(df,accounts_df[['meter_uuid','account_uuid','zip5']],on='meter_uuid')
                df['date'] = pd.to_datetime(df['date'], unit='s')
                try:
                    df.to_sql(name='meter_data', con=engine, if_exists='append', index=False)
                except:
                    print "failed sql insert. meter_uuid:" + str(df['meter_uuid'][0]) + ", filename: " + f.split("_Pacific")[0] + "..."

failed sql insert. meter_uuid:8496493494, filename: 01100176106751_20122013...
failed sql insert. meter_uuid:5637199274, filename: 01612340135426_20122013...
failed sql insert. meter_uuid:5741361055, filename: 04615230123687_20122013...
failed sql insert. meter_uuid:1626781530, filename: 06616220630038_20122013...
failed sql insert. meter_uuid:6449693283, filename: 10739651030402_20122013...
failed sql insert. meter_uuid:64274618, filename: 12755151232107_20122013...
failed sql insert. meter_uuid:5158002781, filename: 15101571530302_20122013...
failed sql insert. meter_uuid:5158002781, filename: 15101571530310_20122013...
failed sql insert. meter_uuid:5158002772, filename: 15101576069496_20122013...
failed sql insert. meter_uuid:7689247028, filename: 20652012030047_20122013...
failed sql insert. meter_uuid:7689247028, filename: 20652012032357_20122013...
failed sql insert. meter_uuid:9227619884, filename: 29663572930048_20122013...
failed sql insert. meter_uuid:8417601597, filename: 29

### Quick tests to make sure it's done so properly

In [22]:
pd.read_sql('SELECT * FROM meter_data LIMIT 3;', con=engine)

,meter_uuid,account_uuid,date,zip5,h1,h2,h3,h4,h5,h6,...,h87,h88,h89,h90,h91,h92,h93,h94,h95,h96
0,1021789005,44697326049563,2014-01-07,95060,2880,0,6160,2720,2240,2480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1021789379,44697326049563,2014-01-08,95060,880,880,960,880,800,880,...,1600.0,1680.0,1600.0,1760.0,1600.0,1600.0,1360.0,1120.0,1120.0,1280.0
2,1021789379,44697326049563,2014-01-09,95060,1040,1440,1120,1200,1120,1120,...,960.0,720.0,880.0,880.0,880.0,880.0,800.0,960.0,880.0,880.0


In [23]:
pd.read_sql('SELECT COUNT(*) FROM meter_data;', con=engine)

,COUNT(*)
0,1286757


### Coordinating between tables to make sure they match

In [24]:
conn.execute('DELETE FROM account WHERE meter_uuid NOT IN (SELECT DISTINCT(meter_uuid) FROM meter_data);')

In [26]:
pd.read_sql('SELECT COUNT(*) FROM account;', con=engine)

,COUNT(*)
0,2186


### Minor cludge that could be done better

This meter_uuid had only one meter_data day record associated with it for some reason, which would cause errors if allowed to stay in the database.

In [27]:
conn.execute("DELETE FROM account WHERE meter_uuid = '1021789005';")
conn.execute("DELETE FROM meter_data WHERE meter_uuid = '1021789005';")